In [ ]:
import numpy as np

from pylib.analyze_additive import (
    assay_additive_naive,
    pick_doses_extrema,
)
from pylib.modelsys_explicit import GenomeExplicit
from pylib.modelsys_explicit import (
    GenomeExplicit,
    CalcKnockoutEffectsAdditive,
    create_additive_array,
)


In [ ]:
np.random.seed(1234)


## Set Up Genome


Create a genome with 1,000 distinct sites, with 5% having a knockout fitness effect below detectability threshold.
Effect sizes are distributed uniformly between 0 and 0.2, relative to the detectability threshold of 1.0.
Knockout effects are assumed additive.


In [ ]:
num_sites = 1000
distn = lambda x: np.random.rand(x) * 0.2  # mean effect size of 0.1
additive_array = create_additive_array(num_sites, 0.05, distn)  # 50 sites
genome = GenomeExplicit(
    [CalcKnockoutEffectsAdditive(additive_array)],
)


For comparison, the true number of additive sites is


In [ ]:
num_additive_sites = additive_array.astype(bool).sum()
num_additive_sites


## Estimate Additive Sites


Perform exploratory knockout experiments at a broad range of dose levels to decide the knockout doses to focus on testing.


In [ ]:
knockout_doses = pick_doses_extrema(
    genome.test_knockout, num_sites, max_doses=5, smear_count=250
)
knockout_doses


Do 1,000 knockout tests at each dose and calculate estimate based on the distribution of sensitivity rates at each dose.


In [ ]:
est = assay_additive_naive(
    genome.test_knockout, num_sites, knockout_doses, num_replications=1000
)
display(est)


In [ ]:
print("actual", num_additive_sites)
print("estimated", est["num additive sites"])
